---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan","Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State","RegionName"]  )'''
    with open('university_towns.txt') as file:
        data = []
        for line in file:
            data.append(line[:-1])
    state_town = []
    for line in data:
        if line[-6:] == '[edit]':
            state = line[:-6]
        elif '(' in line:
            town = line[:line.index('(')-1]
            state_town.append([state,town])
        else:
            town = line
            state_town.append([state,town])
    state_college_df = pd.DataFrame(state_town,columns = ['State','RegionName'])
    return state_college_df
        

get_list_of_university_towns()



,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [4]:
x = pd.ExcelFile('gdplev.xls')
gdp = x.parse(skiprows=7)#skiprows=17,skip_footer=(38))
gdp = gdp[['Unnamed: 4', 'Unnamed: 5']]
gdp.columns = ['Quarter','GDP']
gdp['GDP'] = pd.to_numeric(gdp['GDP'])
gdp

,Quarter,GDP
0,1947q1,243.1
1,1947q2,246.3
2,1947q3,250.1
3,1947q4,260.3
4,1948q1,266.2
5,1948q2,272.9
6,1948q3,279.5
7,1948q4,280.7
8,1949q1,275.4
9,1949q2,271.7


In [5]:
gdp_raw = gdp
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    gdp_filter = gdp_raw.loc[212:]

    quarters = []
    for i in range(len(gdp_filter) - 2):
        if (gdp_filter.iloc[i][1] > gdp_filter.iloc[i+1][1]) & (gdp_filter.iloc[i+1][1] > gdp_filter.iloc[i+2][1]):
            quarters.append(gdp_filter.iloc[i][0])
    return quarters[0]

get_recession_start()

'2008q3'

In [6]:
gdp_raw2 = gdp
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    gdp = gdp_raw2.loc[246:]   
    quarters = []
    for i in range(len(gdp) - 2):
        if (gdp.iloc[i][1] < gdp.iloc[i+1][1]) & (gdp.iloc[i+1][1] < gdp.iloc[i+2][1]):
            quarters.append(gdp.iloc[i][0])
# the first 2 quarters in the list will be connsecutive increase in  GDP, so in the 3rd quarter in the list will be recession end
    return quarters[2]

get_recession_end()

'2009q4'

In [7]:
end = get_recession_end()
start = get_recession_start()
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    rec_start = gdp[gdp['Quarter'] == start]
    i = rec_start.index[0] # of type int64
    rec_end = gdp[gdp['Quarter'] == end]
    j = rec_end.index[0] 
    test_list = []
    gdp_req_series = gdp[i:j]['GDP']
    min_index = gdp_req_series.idxmin(axis = 1)
    return gdp.loc[min_index][0]

get_recession_bottom()

'2009q2'

In [8]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    # following is the code for finding indeces of columns
    #     test = []
    #     for i in range(len(house_df.columns)):
        
    #         test.append(house_df.columns[i] + ' ' + str(i))
    import pandas as pd
    house_df = pd.read_csv('City_Zhvi_AllHomes.csv')
    house_df = house_df.sort_values(by = 'State', axis=0, ascending=True)
    house_df = house_df.set_index(['State','RegionName'])
    cols_drop = []
    for i in range(len(house_df.columns[:49])):
        cols_drop.append(house_df.columns[:49][i])
    house_df = house_df.drop(labels=cols_drop, axis=1)
    date_time_cols = pd.to_datetime(list(house_df.columns)) # converting columns to datetime
    house_df.columns = date_time_cols
    house_df = house_df.resample('Q', axis=1).mean() # converting into quarters
    house_df = house_df.rename(columns=lambda x: str(x.to_period('Q')).lower()) # convert datetime to strings


    return house_df

convert_housing_data_to_quarters()

2000q1         2000q2         2000q3  \
State RegionName                                                      
AK    Kenai                       NaN            NaN            NaN   
      Ketchikan                   NaN            NaN            NaN   
      Anchor Point                NaN            NaN            NaN   
      Anchorage         174633.333333  175266.666667  179566.666667   
      Palmer            168766.666667  170566.666667  174233.333333   
      Seward                      NaN            NaN            NaN   
      Kodiak                      NaN            NaN            NaN   
      Tanaina           160500.000000  163500.000000  166366.666667   
      Lakes             172733.333333  174133.333333  177266.666667   
      North Pole        157466.666667  159100.000000  162133.333333   
      Homer                       NaN            NaN            NaN   
      Soldotna                    NaN            NaN            NaN   
      Juneau            192466.666667  194300.000000  195166.666667   
      Fairbanks         163200.000000  165033.333333  169300.000000   
AL    Birmingham         54033.333333   54400.000000   54966.666667   
      Gurley                      NaN            NaN            NaN   
      Fosters           107500.000000  110200.000000  111400.000000   
      Hope Hull         111666.666667  111800.000000  112966.666667   
      Lipscomb                    NaN            NaN            NaN   
      Guntersville       92933.333333   93500.000000   93600.000000   
      Meadowbrook       200500.000000  203000.000000  206800.000000   
      Northport         125866.666667  126366.666667  126633.333333   
      Morris            124066.666667  124500.000000  124233.333333   
      Good Hope                   NaN            NaN            NaN   
      Orange Beach      221633.333333  222133.333333  224033.333333   
      Robertsdale        86833.333333   87666.666667   89000.000000   
      Cottondale        100833.333333  102633.333333  104766.666667   
      Titus                       NaN            NaN            NaN   
      Leeds              93833.333333   94433.333333   95066.666667   
      Homewood          158333.333333  159666.666667  161100.000000   
...                               ...            ...            ...   
WI    Erin              208566.666667  209100.000000  209666.666667   
WV    Nitro                       NaN            NaN            NaN   
      Eskdale                     NaN            NaN            NaN   
      Mineral Wells      75533.333333   77233.333333   81966.666667   
      Cross Lanes        78133.333333   78066.666667   79233.333333   
      Dunbar                      NaN            NaN            NaN   
      Vienna             72266.666667   74533.333333   75333.333333   
      Saint Albans       63833.333333   64700.000000   65200.000000   
      South Charleston            NaN            NaN            NaN   
      Parkersburg        54900.000000   55833.333333   56633.333333   
      Chesapeake                  NaN            NaN            NaN   
      East Bank                   NaN            NaN            NaN   
      Belle              51066.666667   51300.000000   52200.000000   
      Marmet                      NaN            NaN            NaN   
      Upper Falls        53033.333333   52333.333333   52100.000000   
      Waverly            62366.666667   62933.333333   65300.000000   
      Sissonville        57200.000000   58200.000000   61200.000000   
      Charleston         59533.333333   59366.666667   59366.666667   
      Clendenin          39966.666667   38200.000000   40500.000000   
      Washington         83900.000000   86033.333333   88333.333333   
      Williamstown       82933.333333   85200.000000   86466.666667   
      Davisville                  NaN            NaN            NaN   
      Coal Fork                   NaN            NaN            NaN   
      Elkview                     NaN            NaN   

In [9]:
from scipy import stats
towns_univ = get_list_of_university_towns()
housing_data = convert_housing_data_to_quarters()
rec_bottom = get_recession_bottom()
rec_start = get_recession_start()

def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''

# code for calculating previous quarter
    prev_quarter = []
    for i in range(len(housing_data.columns)):
        if housing_data.columns[i] == rec_start:
            prev_quarter.append(housing_data.columns[i-1]) 
    
    housing_data['Price Ratio'] = housing_data[prev_quarter[0]].div(housing_data[rec_bottom])
    
    # finding university towns
    housing_data_reset = housing_data.reset_index()
    housing_data_reset['State'] = [states[state] for state in housing_data_reset['State']]
    new_df = pd.merge(housing_data_reset, towns_univ, on=towns_univ.columns.tolist(),
        indicator='_flag',how='outer')
    univ_list = new_df[new_df['_flag'] == 'both']
    non_univ_list = new_df[new_df['_flag'] != 'both']
    
#     evaluating which of the towns(university town, non-university town) have lower Price Ratio
#     The one which has lower price ratio is less effected by recession as compared to the other towns
    
    if univ_list['Price Ratio'].mean() < non_univ_list['Price Ratio'].mean():
        better = 'university town'
    else:
        better = 'non-university town'
        
   # finding out the p-value     
    
    p_val = stats.ttest_ind(univ_list['Price Ratio'],non_univ_list['Price Ratio'], nan_policy='omit')[1]
    
# f pvalue <0.01 we reject the null hypothesis and hence it's possible that 
# there is a relationship between {housing prices being affected by recession} and {being located in a university town or not a university town}

# if pvalue >0.01 we cannot reject the null hypothesis and hence there 
# is no relationship between {housing prices being affected by recession} and {being located in a university town or not a university town}
  
    if p_val < 0.01:
        different = True
    else:
        different = False


    return (different,p_val,better)
run_ttest()

(True, 0.0027240637047531249, 'university town')